# RAG application with Azure Open AI & Azure Cognitive Search
## French legal usecase

<img src="https://github.com/retkowsky/images/blob/master/azure_openai_logo.png?raw=true" width=400>

### Objective
Let's build an application that will analyse **legal PDF documents using a RAG application.**

**Retrieval-Augmented Generation (RAG)** can exhibit variability in its implementation, but at a fundamental level, employing RAG within an AI-driven application involves the following sequential steps:

- The user submits a query or question.
- The system initiates a search for pertinent documents that hold the potential to address the user's query. These documents are often comprised of proprietary data and are maintained within a document index.
- The system formulates an instruction set for the Language Model (LLM) that encompasses the user's input, the identified relevant documents, and directives on how to utilize these documents to respond to the user's query effectively.
- The system transmits this comprehensive prompt to the Language Model.
- The Language Model processes the prompt and generates a response to the user's question, drawing upon the context provided. This response constitutes the output of our system.

### Steps
- Uploading PDF documents into an Azure Cognitive Search Index
- Use of some Azure Cognitive Search queries to get some answers
- Use a GPT model to analyse the answer (summmary, keywords generation)
- Get the text from the document and the reference to validate the proposed answer
- Chatbot experience using Azure Open AI to ask questions and get results provided by AI with references

### Process
<img src="https://github.com/retkowsky/images/blob/master/rag.png?raw=true" width=800>

In [1]:
# %pip install azure-search-documents==11.4.0b8

In [2]:
#import azure.cognitiveservices.speech as speechsdk
import datetime
import gradio as gr
#import json
import langchain
import openai
import os
import sys
import time

from azure_rag import (
    get_storage_info,
    get_stats_from_pdf_file,
    delete_index,
    ask_gpt,
    index_status,
    index_stats,
    openai_text_embeddings,
    #chunking,
    upload_docs,
    similarity_comparison,
    azure_openai,
    ask_your_own_data,
)
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

In [3]:
print(f"Today: {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today: 31-Oct-2023 04:29:56


In [4]:
get_storage_info()

Total storage: 126.8 GB
- Used: 115.8 GB    91.37%
- Free: 10.9 GB     8.62%


## 1. Settings

In [5]:
print(f"Python version: {sys.version}")
print(f"OpenAI version: {openai.__version__}")
print(f"Langchain version: {langchain.__version__}")

Python version: 3.10.10 (main, Mar 21 2023, 18:45:11) [GCC 11.2.0]
OpenAI version: 0.28.1
Langchain version: 0.0.300


In [6]:
load_dotenv("azure.env")

# Azure Open AI
openai.api_type: str = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

# Azure Cognitive Search
azure_cs_endpoint = os.getenv("AZURE_COGNITIVE_SEARCH_ENDPOINT")
azure_cs_key = os.getenv("AZURE_COGNITIVE_SEARCH_API_KEY")

In [7]:
# Azure Open AI models (Should be deployed on your Azure OpenAI studio)
embed_model: str = "text-embedding-ada-002"  # Embedding model
gptmodel: str = "gpt-35-turbo-16k"  # GPT Model

# Azure Cognitive search index
index_name: str = "french-penal-code-rag"

In [8]:
print(
    f"We will use {embed_model} as the embedding model and {gptmodel} as the Azure Open AI model"
)

We will use text-embedding-ada-002 as the embedding model and gpt-35-turbo-16k as the Azure Open AI model


In [9]:
print(f"We will create the Azure Cognitive Search index: {index_name}")

We will create the Azure Cognitive Search index: french-penal-code-rag


## 2. PDF documents

In [10]:
PDF_DIR = "documents"

pdf_files = [file for file in os.listdir(PDF_DIR) if file.lower().endswith(".pdf")]
pdf_files

['code_justice_penale_mineurs.pdf',
 'code_penal.pdf',
 'code_procedure_penale.pdf',
 'Introduction_au_droit.pdf']

In [11]:
!ls $PDF_DIR/*.pdf -lh

-rwxrwxrwx 1 root root 289K Oct 26 07:16 documents/Introduction_au_droit.pdf
-rwxrwxrwx 1 root root 2.9M Oct 26 07:16 documents/code_justice_penale_mineurs.pdf
-rwxrwxrwx 1 root root 6.0M Oct 26 07:16 documents/code_penal.pdf
-rwxrwxrwx 1 root root  20M Oct 26 07:16 documents/code_procedure_penale.pdf


## 3. Loading the French PDF documents

We will chunk our PDF documents, do the embeddings and save the content into Azure Cognitive Search

In [12]:
embed_model

'text-embedding-ada-002'

In [15]:
# Embeddings engine
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(engine=embed_model)

In [14]:
# Azure Cognitive Search as the vector store
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=azure_cs_endpoint,
    azure_search_key=azure_cs_key,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

### Testing the Azure GPT model

In [16]:
messages_history = []
messages_history = ask_gpt("Hello", messages_history, gptmodel)

Hello! How can I assist you today? [{'role': 'user', 'content': 'Hello'}]


In [17]:
messages_history = ask_gpt("What can you do?", messages_history, gptmodel)

As an AI, I can do a variety of things. I can answer questions, provide information, help with tasks such as setting reminders or timers, play games, engage in conversation, offer suggestions, and much more. Is there something specific you would like assistance with? [{'role': 'user', 'content': 'Hello'}, {'role': 'user', 'content': 'What can you do?'}]


### Testing the embedded model

In [18]:
prompt1 = "My name is James Bond"
prompt2 = "Ian Fleming"
prompt3 = "Azure Open AI"

In [19]:
emb1 = openai_text_embeddings(prompt1, embed_model)
emb2 = openai_text_embeddings(prompt2, embed_model)
emb3 = openai_text_embeddings(prompt3, embed_model)

In [20]:
emb1[:10]

[-0.03617486730217934,
 -0.005520837381482124,
 -0.007070655468851328,
 -0.030174769461154938,
 0.0020399712957441807,
 0.029502559453248978,
 -0.016581185162067413,
 -0.010979433543980122,
 -0.024423636496067047,
 -0.006939948070794344]

In [21]:
len(emb1)

1536

In [22]:
print("Similarity between:", prompt1, "and", prompt1)
similarity_comparison(emb1, emb1)

Similarity between: My name is James Bond and My name is James Bond
❤️ IDENTICAL text (cosine similarity = 1.0)


In [23]:
print("Similarity between:", prompt1, "and", prompt2)
similarity_comparison(emb1, emb2)

Similarity between: My name is James Bond and Ian Fleming
👍 SIMILAR SEMANTIC text (cosine similarity = 0.827569536463176)


In [24]:
print("Similarity between:", prompt1, "and", prompt3)
similarity_comparison(emb1, emb3)

Similarity between: My name is James Bond and Azure Open AI
🔥 DIFFERENT text (cosine similarity = 0.719675760796164)


In [25]:
print("Similarity between:", prompt2, "and", prompt3)
similarity_comparison(emb2, emb3)

Similarity between: Ian Fleming and Azure Open AI
🔥 DIFFERENT text (cosine similarity = 0.727620333588871)


### Processing the PDF docs

In [26]:
!ls $PDF_DIR -lh

total 29M
-rwxrwxrwx 1 root root 289K Oct 26 07:16 Introduction_au_droit.pdf
-rwxrwxrwx 1 root root 2.9M Oct 26 07:16 code_justice_penale_mineurs.pdf
-rwxrwxrwx 1 root root 6.0M Oct 26 07:16 code_penal.pdf
-rwxrwxrwx 1 root root  20M Oct 26 07:16 code_procedure_penale.pdf


In [27]:
for i in range(len(pdf_files)):
    # Getting stats for each file
    filepath = os.path.join(PDF_DIR, pdf_files[i])
    !ls $filepath -lh
    get_stats_from_pdf_file(filepath)

-rwxrwxrwx 1 root root 2.9M Oct 26 07:16 documents/code_justice_penale_mineurs.pdf
documents/code_justice_penale_mineurs.pdf:
- Number of pages = 166
- Number of words = 69698        
- Number of tokens = 141485

-rwxrwxrwx 1 root root 6.0M Oct 26 07:16 documents/code_penal.pdf
documents/code_penal.pdf:
- Number of pages = 419
- Number of words = 180511        
- Number of tokens = 364049

-rwxrwxrwx 1 root root 20M Oct 26 07:16 documents/code_procedure_penale.pdf
documents/code_procedure_penale.pdf:
- Number of pages = 1683
- Number of words = 702687        
- Number of tokens = 1385267

-rwxrwxrwx 1 root root 289K Oct 26 07:16 documents/Introduction_au_droit.pdf
documents/Introduction_au_droit.pdf:
- Number of pages = 144
- Number of words = 57925        
- Number of tokens = 102209



In [28]:
start = time.time()
print("Documents processing...\n")

for i in range(len(pdf_files)):
    print(f"Uploading file {i + 1}")
    # Chunking and loading into the Azure Cognitive Search index
    upload_docs(PDF_DIR, pdf_files[i], vector_store)
    time.sleep(5)

elapsed = time.time() - start
print("\nDone")
print(
    "Elapsed time: "
    + time.strftime(
        "%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:15], time.gmtime(elapsed)
    )
)

Documents processing...

Uploading file 1
Processing the document documents/code_justice_penale_mineurs.pdf
Chunking the document...
Loading the embeddings into Azure Cognitive Search...
Done

Uploading file 2
Processing the document documents/code_penal.pdf
Chunking the document...
Loading the embeddings into Azure Cognitive Search...
Done

Uploading file 3
Processing the document documents/code_procedure_penale.pdf
Chunking the document...
Loading the embeddings into Azure Cognitive Search...


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Get a vector representation of a given input that can be easily consumed by machine learning models and algorithms. Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 9 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Get a vector representation of a given input that can be easily consumed by machine learning models and algorithms. Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 5 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to 

Done

Uploading file 4
Processing the document documents/Introduction_au_droit.pdf
Chunking the document...
Loading the embeddings into Azure Cognitive Search...
Done


Done
Elapsed time: 00:06:32.322322


## 4. Azure Cognitive Search index status

Our index is available and ready to use

In [29]:
print(f"Azure Cognitive Search index = {index_name}")

Azure Cognitive Search index = french-penal-code-rag


In [30]:
index_status(index_name, azure_cs_endpoint, azure_cs_key)

Azure Cognitive Search Index: french-penal-code-rag 

{
     "@odata.context": "https://azurecogsearcheastussr.search.windows.net/$metadata#indexes/$entity",
     "@odata.etag": "\"0x8DBD9CA0FE2C4C2\"",
     "name": "french-penal-code-rag",
     "defaultScoringProfile": null,
     "fields": [
          {
               "name": "id",
               "type": "Edm.String",
               "searchable": false,
               "filterable": true,
               "retrievable": true,
               "sortable": false,
               "facetable": false,
               "key": true,
               "indexAnalyzer": null,
               "searchAnalyzer": null,
               "analyzer": null,
               "normalizer": null,
               "synonymMaps": []
          },
          {
               "name": "content",
               "type": "Edm.String",
               "searchable": true,
               "filterable": false,
               "retrievable": true,
               "sortable": false,
         

In [31]:
document_count, storage_size = index_stats(index_name, azure_cs_endpoint, azure_cs_key)

Azure Cognitive Search index status for: french-penal-code-rag 

{
  "@odata.context": "https://azurecogsearcheastussr.search.windows.net/$metadata#Microsoft.Azure.Search.V2021_04_30_Preview.IndexStatistics",
  "documentCount": 493,
  "storageSize": 15082473
}


In [32]:
print(f"Number of documents in the index = {document_count}")
print(f"Size of the index = {round(storage_size / (1024 * 1024), 2)} MB")

Number of documents in the index = 493
Size of the index = 14.38 MB


## 5. Search with Azure Open AI

### 5.1 Testing

In [39]:
%%javascript Python
OutputArea.auto_scroll_threshold = 9999

<IPython.core.display.Javascript object>

This is to maximize the display output

We can do some quick tests

In [40]:
print(f"Azure OpenAI model to use: {gptmodel}")

Azure OpenAI model to use: gpt-35-turbo-16k


In [41]:
print(azure_openai("What time is it?", gptmodel))

I'm sorry, but I don't have access to real-time information.


In [42]:
print(azure_openai("Who are you?", gptmodel))

I am a legal expert assistant. I can help answer questions related to legal topics.


In [43]:
print(azure_openai("Qui es-tu?", gptmodel))

Je suis un assistant expert en droit.


In [44]:
print(azure_openai("What can you do for me?", gptmodel))

I can assist you with legal information and provide answers to your legal questions based on the given context. I can extract and print article references, summarize the text, and extract keywords from the text. Please provide me with the specific context or question you would like assistance with.


### 5.2 Ask your own data 

We can ask any questions related with the documents we have processed. You can ask in any language.

In [52]:
query = "How about identity checks in criminal cases?"

ask_your_own_data(query, vector_store, gptmodel, topn=3)


Azure Open AI results:

 1. Articles: 62, 78-1, 78-2, 78-3-1, 55, 55-1
2. Summary: The text discusses the conduct of criminal investigations and identity checks in the French legal system. It mentions the conditions and procedures for conducting investigations, including the questioning of witnesses and the drafting of reports. It also explains the rules for identity checks, including the reasons for conducting them and the rights of the individuals being checked. The text also mentions specific articles and decisions related to these topics.
3. Keywords: conduct, criminal investigations, identity checks, witnesses, reports, questioning, rules, rights, articles, decisions.

[Note] This summary is generated by an AI (Azure Open AI). Examine and use carefully.

Document reference 1:

Partie législative - Livre Ier : De la conduite de la politique pénale, de l'exercice de l'action publique et de l'instruction - Titre II : Des enquêtes et des contrôles d'identité
de risque de modification

In [46]:
query = "What are the French laws regarding corruption?"

ask_your_own_data(query, vector_store, gptmodel, topn=3)


Azure Open AI results:

 1. Articles: 435-7, 435-8, 435-9, 435-1, 435-2, 435-3, 445-1, 445-1-1, 445-2
2. Summary: The text discusses various crimes related to corruption and influence trafficking. It outlines the penalties for individuals who engage in passive corruption and influence trafficking, such as soliciting or accepting offers, promises, gifts, or any other advantages to abuse their real or supposed influence. It also covers active corruption and influence trafficking, which involves offering or proposing offers, promises, gifts, or any other advantages to individuals in positions of authority or public service. The text specifies the penalties for these offenses, including imprisonment and fines. Additionally, it addresses corruption and influence trafficking by individuals who do not hold public office, but who exercise a leadership role or work for an organization. The penalties for these offenses are also outlined. 
3. Keywords: corruption, influence trafficking, passive 

In [47]:
query = "Quelle est la peine encourue pour un vol ?"

ask_your_own_data(query, vector_store, gptmodel, topn=3)


Azure Open AI results:

 1. Articles: 311-6, 311-7, 311-8, 311-9, 311-9-1, 311-4-1, 311-4-2, 311-5, 434-35-1, 434-36, 434-37, 434-38, 434-38-1, 434-39
2. Summary: The text discusses the penalties for theft and related offenses. It specifies the punishments for different circumstances, such as theft accompanied by violence, theft in organized gangs, theft with the use of weapons, and theft in specific locations or situations. The penalties range from imprisonment for a certain number of years to fines. The text also mentions the reduction of penalties for individuals who report or prevent the commission of theft. Additionally, it addresses offenses related to the authority of the state and the justice system, such as unauthorized entry into a prison and the removal or destruction of court-ordered posters. 
3. Keywords: theft, penalties, imprisonment, fines, violence, organized gangs, weapons, specific locations, unauthorized entry, destruction of posters

[Note] This summary is generat

In [48]:
query = "Quelles sont les missions du procureur général ?"

ask_your_own_data(query, vector_store, gptmodel, topn=3)


Azure Open AI results:

 1. Articles: 33 Loi 57-1426 1957-12-31, 34 LOI n° 2014-1170 du 13 octobre 2014, 35 LOI n°2013-669 du 25 juillet 2013, 36 Loi n°2004-204 du 9 mars 2004, 37 Loi n°2004-204 du 9 mars 2004, 39 Ordonnance n°2019-964 du 18 septembre 2019, 39-1 LOI n°2013-669 du 25 juillet 2013, 39-2 Ordonnance n°2019-964 du 18 septembre 2019, 39-3 LOI n°2016-731 du 3 juin 2016, 39-4 LOI n°2019-222 du 23 mars 2019, 40 Loi n°2004-204 du 9 mars 2004, 40-1 LOI n°2016-1691 du 9 décembre 2016
2. Summary: This text is part of the French Code of Criminal Procedure and specifically focuses on the roles and responsibilities of the prosecutor general and the prosecutor of the Republic. The prosecutor general represents the public ministry before the court of appeal and other courts. They ensure the application of criminal law and coordinate the actions of the prosecutors of the Republic. They also provide reports on the implementation of criminal policy. The prosecutor of the Republic represen

In [49]:
query = "¿Qué es el artículo 131-39 del Código Penal?"

ask_your_own_data(query, vector_store, gptmodel, topn=1)


Azure Open AI results:

 1. Articles: 223-15-2, 223-15-3, 223-15-4
2. Summary: The text discusses the punishment for fraudulent abuse of ignorance or weakness. It states that anyone who fraudulently takes advantage of the ignorance or vulnerability of a minor or a person with particular vulnerability due to age, illness, disability, pregnancy, or psychological or physical subjection can be sentenced to three years in prison and a fine of 375,000 euros. The punishment is increased to five years in prison and a fine of 750,000 euros if the offense is committed by the leader of a group that maintains psychological or physical subjection. If the offense is committed by members of an organized group, the punishment is increased to seven years in prison and a fine of one million euros. The text also mentions additional penalties such as the prohibition of certain rights, the closure of establishments used to commit the offense, confiscation of relevant objects, and the prohibition of certai

In [50]:
query = "What is a crime against the nation?"

ask_your_own_data(query, vector_store, gptmodel, topn=1)


Azure Open AI results:

 1. Articles: 413-3, 413-4, 413-5, 413-6, 413-7
2. Summary: The text refers to the punishment for actions that aim to harm national defense. It states that obstructing the movement of military personnel or equipment, provoking disobedience among military personnel, participating in activities to demoralize the army, unlawfully entering military premises, and impeding the normal functioning of public or private services related to national defense are all punishable offenses. The penalties for these offenses range from one year imprisonment and a fine of 15,000 euros to five years imprisonment and a fine of 75,000 euros.
3. Keywords: defense nationale (national defense), mouvement de personnel (movement of personnel), matériel militaire (military equipment), désobéissance (disobedience), démoralisation de l'armée (demoralization of the army), s'introduire frauduleusement (unlawfully entering), fonctionnement normal des services (normal functioning of services), 

In [51]:
query = "What are the penalties for forgery?"

ask_your_own_data(query, vector_store, gptmodel, topn=1)


Azure Open AI results:

 1. Articles: 441-5, 441-6, 441-7, 441-8
2. Summary: The text discusses various provisions related to false documents and fraud in the context of public administration. It mentions the penalties for committing forgery or using false documents by a public official or someone acting in a public service capacity. It also mentions the penalties for fraudulently providing a document issued by a public administration, as well as the penalties for obtaining or attempting to obtain benefits through false declarations. Additionally, it mentions the penalties for establishing or using false certificates or attestations. Finally, it mentions the penalties for using someone else's identity or documents to enter or stay in France illegally or to obtain undeserved benefits. 
3. Keywords: peines (penalties), réclusion criminelle (criminal imprisonment), amende (fine), faux (forgery), usage de faux (use of false documents), personne dépositaire de l'autorité publique (person h

## 6. Gradio webapp

Let's build a quick webapp with gradio to test our RAG application

In [53]:
def legal_rag_function(query):
    """
    RAG function with Azure Cognitive Search and Azure Open AI
    Input: query
    Output: result list with the results for a gradio webapp
    """
    # Warning message
    msg = "\n\n[Note] This summary is generated by an AI (powered by Azure Open AI). Examine and use carefully."
    
    # Calling Azure Cognitive Search
    results = vector_store.similarity_search_with_relevance_scores(
        query=query,
        k=5,
        score_threshold=0.5,
    )

    fulltext_list = []

    for idx in range(5):
        reference = results[idx][0].__dict__["page_content"]
        fulltext_list.append(reference)

    # Get the AOAI summary
    fulltext = "".join(fulltext_list)
    answer = azure_openai(fulltext, gptmodel)
    
    # Lists declaration
    results_list = []
    source_list = []
    confidence_list = []

    # Output generation
    for i in range(5):
        # Get Azure Cognitive Search result
        reference = results[i][0]
        ref = reference.__dict__
        ref = ref["page_content"]

        # Get source information
        source = reference.metadata
        doc_source = source["source"]
        page_source = source["page"]
        confidence = results[i][1]

        source_ref = f"Source: {doc_source} Page: {page_source}"
        source_list.append(source_ref)
        confidence_list.append(confidence)

        # Adding all the results into a single list
        results_list.append(ref)
        results_list.append(source_list[i])
        results_list.append(confidence_list[i])

    results_list.insert(0, answer + msg)  # Insert the answer on the top of the list

    return results_list

### Webapp function definition

In [54]:
image_url = "https://github.com/retkowsky/images/blob/master/legal.jpg?raw=true"
image = "<center> <img src= {} width=200px></center>".format(image_url)
header = "RAG application with Azure Open AI & Azure Cognitive Search (French penal code example)"

samples = [
    "How about identity checks in France?",
    "What is the definition of a crime against humanity?",
    "What are the penalties for fraud?",
    "Article 131-39 of the penal code",
    "Quid des frais de justice ?",
    "Quelles sont les missions du procureur général ?",
]

inputs = gr.Text(type="text", label="Question:", lines=5, max_lines=10)

outputs = [
    gr.Text(label="Azure Open AI summary"),
    gr.Text(label="1. Reference"),
    gr.Text(label="1. Document source"),
    gr.Number(label="1. Confidence"),
    gr.Text(label="2. Reference"),
    gr.Text(label="2. Document source"),
    gr.Number(label="2. Confidence"),
    gr.Text(label="3. Reference"),
    gr.Text(label="3. Document source"),
    gr.Number(label="3. Confidence"),
    gr.Text(label="4. Reference"),
    gr.Text(label="4. Document source"),
    gr.Number(label="4. Confidence"),
    gr.Text(label="5. Reference"),
    gr.Text(label="5. Document source"),
    gr.Number(label="5. Confidence"),
]

theme = "freddyaboulton/test-blue"
# https://huggingface.co/spaces/gradio/theme-gallery

legal_rag_webapp = gr.Interface(
    fn=legal_rag_function,
    inputs=inputs,
    outputs=outputs,
    description=image,
    title=header,
    examples=samples,
    theme=theme,
)

### Running the webapp

In [55]:
legal_rag_webapp.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://0503e311e6cfe8fc44.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


## 7. Post Processing

We can delete our index if needed

In [56]:
index_status(index_name, azure_cs_endpoint, azure_cs_key)

Azure Cognitive Search Index: french-penal-code-rag 

{
     "@odata.context": "https://azurecogsearcheastussr.search.windows.net/$metadata#indexes/$entity",
     "@odata.etag": "\"0x8DBD9CA0FE2C4C2\"",
     "name": "french-penal-code-rag",
     "defaultScoringProfile": null,
     "fields": [
          {
               "name": "id",
               "type": "Edm.String",
               "searchable": false,
               "filterable": true,
               "retrievable": true,
               "sortable": false,
               "facetable": false,
               "key": true,
               "indexAnalyzer": null,
               "searchAnalyzer": null,
               "analyzer": null,
               "normalizer": null,
               "synonymMaps": []
          },
          {
               "name": "content",
               "type": "Edm.String",
               "searchable": true,
               "filterable": false,
               "retrievable": true,
               "sortable": false,
         

In [57]:
delete_index(index_name, azure_cs_endpoint, azure_cs_key)

Deleting the Azure Cognitive Search index: french-penal-code-rag
Done. Elapsed time: 0.69 secs


In [58]:
index_status(index_name, azure_cs_endpoint, azure_cs_key)

Azure Cognitive Search Index: french-penal-code-rag 

{
     "error": {
          "code": "",
          "message": "No index with the name 'french-penal-code-rag' was found in the service 'azurecogsearcheastussr'."
     }
}


> End of notebook